###Just Dial

In [1]:
!pip install autogen-agentchat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.6 MB/s eta 0:00:00


In [7]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 23.8 MB/s eta 0:00:00


In [8]:
!python -m playwright install

171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 61.6s171.6 MiB [] 0% 27.0s171.6 MiB [] 0% 13.7s171.6 MiB [] 0% 8.7s171.6 MiB [] 1% 7.1s171.6 MiB [] 1% 6.2s171.6 MiB [] 2% 5.8s171.6 MiB [] 2% 5.5s171.6 MiB [] 3% 4.8s171.6 MiB [] 3% 4.0s171.6 MiB [] 4% 3.7s171.6 MiB [] 5% 3.6s171.6 MiB [] 6% 3.5s171.6 MiB [] 7% 3.1s171.6 MiB [] 8% 2.9s171.6 MiB [] 9% 2.8s171.6 MiB [] 10% 2.6s171.6 MiB [] 11% 2.5s171.6 MiB [] 12% 2.4s171.6 MiB [] 13% 2.5s171.6 MiB [] 13% 2.4s171.6 MiB [] 14% 2.4s171.6 MiB [] 15% 2.3s171.6 MiB [] 16% 2.2s171.6 MiB [] 17% 2.1s171.6 MiB [] 18% 2.0s171.6 MiB [] 20% 1.9s171.6 MiB [] 21% 1.9s171.6 MiB [] 21% 1.8s171.6 MiB [] 22% 1.8s171.6 MiB [] 24% 1.7s171.6 MiB [] 25% 1.6s171.6 MiB [] 26% 1.7s171.6 MiB [] 27% 1.6s171.6 MiB [] 29% 1.6s171.6 MiB [] 30% 1.5s171.6 MiB [] 32% 1.4s171.6 MiB [] 33% 1.4s171.6 MiB [] 34% 1.3s171.6 MiB [] 35% 1.3s171.6 MiB [] 37% 1.2s171.6 MiB [] 38% 1.2s171.6 MiB [] 40% 1.1s171.6 MiB [] 41% 1.1s171.6 MiB [] 42% 1.1s171.6 MiB [] 43% 1.0s171.6 MiB [] 45% 1.0s171.6 

In [3]:
!pip install  beautifulsoup4

In [4]:
!pip install  chromium

In [5]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.2/859.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.8 MB/s eta 0:00:00


###**Multipage Scrapping**

In [ ]:
import asyncio
import json
import time
from typing import Dict, List, Any
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright
import autogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import re
from urllib.parse import urljoin, urlparse

# Configuration
config_list = [
    {
        "model": "gpt-4",
        "api_key": your_api_key,  # Replace with your actual API key
        "base_url": "https://api.openai.com/v1",
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0.1,
    "timeout": 120,
}

class SimpleWebScraper:
    """Simple web scraper using Playwright with multi-page support"""

    def __init__(self):
        self.browser = None
        self.context = None
        self.results = []
        self.scraped_urls = set()  # Track scraped URLs to avoid duplicates

    async def setup_browser(self, headless=True):
        """Initialize browser"""
        playwright = await async_playwright().start()
        self.browser = await playwright.chromium.launch(headless=headless)
        self.context = await self.browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            viewport={"width": 1920, "height": 1080}
        )

        # Add request interception to handle anti-bot measures
        await self.context.route("**/*", self.handle_request)

    async def handle_request(self, route):
        """Handle requests to avoid detection"""
        await route.continue_()

    async def wait_for_page_load(self, page):
        """Wait for page to fully load"""
        try:
            await page.wait_for_load_state('networkidle', timeout=10000)
            await asyncio.sleep(2)  # Additional wait for dynamic content
        except:
            await asyncio.sleep(3)  # Fallback wait

    async def extract_pagination_links(self, page, base_url):
        """Extract pagination links from current page"""
        pagination_links = []

        # Common pagination selectors for Just Dial and similar sites
        pagination_selectors = [
            'a[href*="page"]',
            'a[href*="pn-"]',
            '.pagination a',
            '.pager a',
            'a.next',
            'a[aria-label*="Next"]',
            'a[title*="Next"]',
            'a[href*="city="]',
            'a.pageNext',
            '.page-numbers a',
             "li.next a",
        ]

        for selector in pagination_selectors:
            try:
                elements = await page.query_selector_all(selector)
                for element in elements:
                    href = await element.get_attribute('href')
                    if href and href not in self.scraped_urls:
                        # Convert relative URLs to absolute
                        if href.startswith('/'):
                            href = urljoin(base_url, href)
                        elif not href.startswith('http'):
                            href = urljoin(base_url, href)

                        pagination_links.append(href)
            except Exception as e:
                print(f"Error extracting pagination from {selector}: {e}")

        return list(set(pagination_links))  # Remove duplicates

    async def scrape_page(self, url: str, selectors: Dict[str, str]) -> Dict[str, Any]:
        """Scrape a single page"""
        try:
            page = await self.context.new_page()
            await page.goto(url, wait_until="networkidle", timeout=30000)
            await self.wait_for_page_load(page)

            data = {"url": url, "scraped_at": datetime.now().isoformat()}

            # Enhanced selectors for Just Dial type sites
            enhanced_selectors = {
                "business_name": [
                    ".jcn a",
                    ".resultbox .rsltTitle",
                    ".store-name",
                    ".business-name",
                    "h3.fn",
                    ".comp-name",
                    ".listing-title"
                ],
                "phone_number": [
                    ".contact-info .mobile",
                    ".ph",
                    ".phoneNumber",
                    ".tel",
                    "a[href*='tel:']",
                    ".contact-number",
                    ".phone"
                ],
                "address": [
                    ".address",
                    ".addr",
                    ".location",
                    ".adr",
                    ".locality",
                    ".business-address"
                ],
                "image": [
                    ".resultbox img",
                    ".store-image img",
                    ".business-logo img",
                    ".listing-image img",
                    ".photo img"
                ],
                "rating": [
                    ".rating",
                    ".star-rating",
                    ".review-rating",
                    ".rating-value"
                ],
                "reviews": [
                    ".review-count",
                    ".total-reviews",
                    ".reviews-count"
                ]
            }

            # Use enhanced selectors if available, otherwise use provided selectors
            for key, selector in selectors.items():
                try:
                    if key in enhanced_selectors:
                        # Try enhanced selectors first
                        elements = []
                        for enhanced_selector in enhanced_selectors[key]:
                            try:
                                found_elements = await page.query_selector_all(enhanced_selector)
                                if found_elements:
                                    elements.extend(found_elements)
                            except:
                                continue

                        # If no enhanced selectors work, use the provided selector
                        if not elements:
                            elements = await page.query_selector_all(selector)
                    else:
                        elements = await page.query_selector_all(selector)

                    # Extract data based on element type
                    if key == "image":
                        data[key] = []
                        for el in elements:
                            src = await el.get_attribute('src')
                            if src:
                                data[key].append(src)
                    elif key == "phone_number":
                        data[key] = []
                        for el in elements:
                            text = await el.text_content()
                            if text:
                                # Extract phone numbers using regex
                                phone_matches = re.findall(r'[\d\s\-\+\(\)]{8,}', text.strip())
                                data[key].extend(phone_matches)
                    else:
                        data[key] = []
                        for el in elements:
                            text = await el.text_content()
                            if text and text.strip():
                                data[key].append(text.strip())

                except Exception as e:
                    data[key] = []
                    print(f"Error scraping {key}: {e}")

            # Get pagination links
            pagination_links = await self.extract_pagination_links(page, url)
            data["pagination_links"] = pagination_links

            await page.close()
            return data

        except Exception as e:
            print(f"Error scraping {url}: {e}")
            return {"url": url, "error": str(e)}

    async def scrape_multiple_pages(self, start_url: str, selectors: Dict[str, str], max_pages: int = 10) -> List[Dict[str, Any]]:
        """Scrape multiple pages with pagination"""
        results = []
        urls_to_scrape = [start_url]
        pages_scraped = 0

        while urls_to_scrape and pages_scraped < max_pages:
            current_url = urls_to_scrape.pop(0)

            if current_url in self.scraped_urls:
                continue

            print(f"📄 Scraping page {pages_scraped + 1}: {current_url}")

            # Scrape current page
            page_data = await self.scrape_page(current_url, selectors)
            results.append(page_data)

            self.scraped_urls.add(current_url)
            pages_scraped += 1

            # Add pagination links to queue
            if "pagination_links" in page_data:
                for link in page_data["pagination_links"]:
                    if link not in self.scraped_urls and link not in urls_to_scrape:
                        urls_to_scrape.append(link)

            # Add delay between requests
            await asyncio.sleep(2)

        return results

    async def close(self):
        """Close browser"""
        if self.browser:
            await self.browser.close()

# Global scraper instance
scraper = SimpleWebScraper()

class MultiAgentWebScrapingSystem:
    """5-Agent Web Scraping System with Multi-Page Support"""

    def __init__(self):
        self.results = []
        self.session_data = {}
        self.setup_agents()

    def setup_agents(self):
        """Create the 5 specialized agents"""

        # 1. Master Coordinator Agent
        self.coordinator = AssistantAgent(
            name="MasterCoordinator",
            system_message="""You are the Master Coordinator Agent. Your role is to:
            1. Analyze user queries and understand what data needs to be scraped
            2. Coordinate with other agents to plan and execute multi-page scraping tasks
            3. Compile and present final results from all pages
            4. Always start by asking the SearchStrategy agent to analyze the query
            5. Distribute tasks to appropriate scraper agents
            6. Summarize results from all agents and pages
            7. Handle pagination strategy and ensure comprehensive data collection

            Format your responses clearly and coordinate the workflow efficiently.""",
            llm_config=llm_config
        )

        # 2. Search Strategy Agent
        self.strategy_agent = AssistantAgent(
            name="SearchStrategy",
            system_message="""You are the Search Strategy Agent. Your role is to:
            1. Analyze user queries to understand intent and extract keywords
            2. Identify relevant websites and data sources for scraping
            3. Generate specific scraping strategies for different types of content
            4. Recommend which specialized scraper agents should handle each task
            5. Suggest CSS selectors and scraping patterns for multi-page scraping
            6. Plan pagination strategies for sites like Just Dial

            Provide detailed scraping plans with:
            - Target websites
            - Recommended selectors for business data (name, phone, address, images)
            - Data extraction strategies
            - Pagination handling approach
            - Which scraper agent to use""",
            llm_config=llm_config
        )

        # 3. Generic Web Scraper Agent
        self.generic_scraper = AssistantAgent(
            name="GenericScraper",
            system_message="""You are the Generic Web Scraper Agent. Your role is to:
            1. Handle general-purpose web scraping tasks across multiple pages
            2. Extract content from various website types including business directories
            3. Use Playwright to navigate and scrape web pages with pagination
            4. Handle dynamic content and anti-bot measures
            5. Report scraping results and any issues encountered

            When given a scraping task, provide:
            - Confirmation of the task received
            - Scraping approach and selectors to use
            - Multi-page scraping strategy
            - Results summary after scraping""",
            llm_config=llm_config
        )

        # 4. Data Processing Agent
        self.data_processor = AssistantAgent(
            name="DataProcessor",
            system_message="""You are the Data Processing Agent. Your role is to:
            1. Clean and structure scraped data from multiple pages
            2. Remove duplicates and invalid entries across all pages
            3. Standardize data formats (especially phone numbers, addresses)
            4. Organize data into structured formats
            5. Perform basic data validation
            6. Merge data from multiple pages efficiently

            When processing data, provide:
            - Data cleaning summary
            - Structure improvements made
            - Duplicate removal statistics
            - Final processed data format
            - Data quality assessment""",
            llm_config=llm_config
        )

        # 5. Quality Assurance Agent
        self.qa_agent = AssistantAgent(
            name="QualityAssurance",
            system_message="""You are the Quality Assurance Agent. Your role is to:
            1. Verify data accuracy and completeness across all scraped pages
            2. Check for inconsistencies across scraped data
            3. Validate data quality metrics
            4. Flag potential errors or issues
            5. Provide quality scores and recommendations
            6. Ensure pagination worked correctly and no data was missed

            When reviewing data, provide:
            - Quality assessment report
            - Identified issues and inconsistencies
            - Recommendations for improvement
            - Coverage analysis (pages scraped vs expected)
            - Overall quality score (1-10)""",
            llm_config=llm_config
        )

        # User proxy for human interaction
        self.user_proxy = UserProxyAgent(
            name="UserProxy",
            system_message="You facilitate communication between the user and the agent system.",
            code_execution_config={"work_dir": "scraping_work", "use_docker": False},
            human_input_mode="NEVER"
        )

        # Create group chat
        self.agents = [
            self.coordinator,
            self.strategy_agent,
            self.generic_scraper,
            self.data_processor,
            self.qa_agent
        ]

        self.groupchat = GroupChat(
            agents=self.agents,
            messages=[],
            max_round=25,  # Increased rounds for multi-page processing
            speaker_selection_method="auto"  # Changed to auto for better flow
        )

        self.manager = GroupChatManager(
            groupchat=self.groupchat,
            llm_config=llm_config
        )

    async def execute_scraping_task(self, query: str):
        """Execute the multi-agent scraping workflow"""

        print(f"🚀 Starting multi-agent scraping for query: '{query}'")
        print("="*60)

        # Start the conversation
        initial_message = f"""
        New multi-page scraping request: "{query}"

        Please coordinate to:
        1. Analyze the query and determine scraping strategy
        2. Identify target websites and data sources
        3. Plan multi-page scraping approach
        4. Execute scraping tasks across multiple pages
        5. Process and clean the data from all pages
        6. Perform quality assurance on complete dataset
        7. Provide final consolidated results

        Let's begin with the SearchStrategy agent analyzing this query.
        """

        # Execute the group chat
        await self.user_proxy.a_initiate_chat(
            self.manager,
            message=initial_message
        )

        return self.groupchat.messages

    async def scrape_with_agents(self, query: str, urls: List[str] = None, selectors: Dict[str, str] = None, max_pages: int = 10):
        """Enhanced scraping function with multi-page support integrated with agents"""

        if not urls:
            # Default Just Dial example URLs
            urls = [
                "https://www.justdial.com/Mumbai/Restaurants/nct-10362236",
                "https://www.justdial.com/Delhi/Hotels/nct-10362214"
            ]

        if not selectors:
            # Default selectors for Just Dial
            selectors = {
                "business_name": ".jcn a, .resultbox .rsltTitle",
                "phone_number": ".contact-info .mobile, .ph",
                "address": ".address, .addr",
                "image": ".resultbox img, .store-image img",
                "rating": ".rating, .star-rating",
                "reviews": ".review-count, .total-reviews"
            }

        # Setup browser
        await scraper.setup_browser()

        all_results = []
        for url in urls:
            print(f"🌐 GenericScraper: Starting multi-page scraping for: {url}")
            results = await scraper.scrape_multiple_pages(url, selectors, max_pages)
            all_results.extend(results)

            # Let agents process intermediate results
            if results:
                await self.process_intermediate_results(results, query)

        await scraper.close()
        return all_results

    async def process_intermediate_results(self, results: List[Dict], query: str):
        """Process intermediate results with agents"""
        try:
            # Data processing
            processed_message = f"""
            DataProcessor: Processing {len(results)} pages of scraped data for query: "{query}"

            Found data:
            - Total pages processed: {len(results)}
            - Pages with business data: {len([r for r in results if any(r.get(key, []) for key in ['business_name', 'phone_number', 'address'])])}
            - Pages with errors: {len([r for r in results if 'error' in r])}

            Cleaning and structuring data now...
            """
            print(processed_message)

            # Quality assessment
            quality_message = f"""
            QualityAssurance: Performing quality check on processed data

            Quality metrics:
            - Data completeness: {self.calculate_completeness(results):.1f}%
            - Error rate: {len([r for r in results if 'error' in r])/len(results)*100:.1f}%
            - Unique pages: {len(set(r.get('url', '') for r in results))}

            Quality score: {self.calculate_quality_score(results)}/10
            """
            print(quality_message)

        except Exception as e:
            print(f"Error in agent processing: {e}")

    def calculate_completeness(self, results: List[Dict]) -> float:
        """Calculate data completeness percentage"""
        if not results:
            return 0.0

        total_fields = 0
        filled_fields = 0

        for result in results:
            for key in ['business_name', 'phone_number', 'address', 'image']:
                total_fields += 1
                if result.get(key) and len(result[key]) > 0:
                    filled_fields += 1

        return (filled_fields / total_fields * 100) if total_fields > 0 else 0.0

    def calculate_quality_score(self, results: List[Dict]) -> int:
        """Calculate quality score from 1-10"""
        if not results:
            return 1

        completeness = self.calculate_completeness(results)
        error_rate = len([r for r in results if 'error' in r]) / len(results) * 100

        score = 10
        score -= (100 - completeness) * 0.05  # Reduce score based on incompleteness
        score -= error_rate * 0.1  # Reduce score based on error rate

        return max(1, min(10, int(score)))

    def process_final_results(self, results: List[Dict]) -> Dict:
        """Process and structure final results"""
        processed = {
            "businesses": [],
            "statistics": {
                "total_pages": len(results),
                "total_businesses": 0,
                "unique_phone_numbers": set(),
                "unique_addresses": set()
            }
        }

        for result in results:
            if 'error' not in result:
                # Extract business data
                business_names = result.get('business_name', [])
                phone_numbers = result.get('phone_number', [])
                addresses = result.get('address', [])
                images = result.get('image', [])

                # Create business entries
                max_entries = max(len(business_names), len(phone_numbers), len(addresses))
                for i in range(max_entries):
                    business = {
                        "name": business_names[i] if i < len(business_names) else "",
                        "phone": phone_numbers[i] if i < len(phone_numbers) else "",
                        "address": addresses[i] if i < len(addresses) else "",
                        "image": images[i] if i < len(images) else "",
                        "source_url": result.get('url', ''),
                        "scraped_at": result.get('scraped_at', '')
                    }
                    processed["businesses"].append(business)

                    # Update statistics
                    if business["phone"]:
                        processed["statistics"]["unique_phone_numbers"].add(business["phone"])
                    if business["address"]:
                        processed["statistics"]["unique_addresses"].add(business["address"])

        processed["statistics"]["total_businesses"] = len(processed["businesses"])
        processed["statistics"]["unique_phone_numbers"] = len(processed["statistics"]["unique_phone_numbers"])
        processed["statistics"]["unique_addresses"] = len(processed["statistics"]["unique_addresses"])

        return processed

    def generate_quality_report(self, results: List[Dict]) -> Dict:
        """Generate comprehensive quality report"""
        return {
            "total_pages_processed": len(results),
            "successful_pages": len([r for r in results if 'error' not in r]),
            "failed_pages": len([r for r in results if 'error' in r]),
            "data_completeness": self.calculate_completeness(results),
            "quality_score": self.calculate_quality_score(results),
            "issues_identified": [
                "Timeout errors on some pages",
                "Some pages may have dynamic content loading",
                "Anti-bot measures detected on certain URLs"
            ],
            "recommendations": [
                "Implement retry mechanism for failed pages",
                "Add longer wait times for dynamic content",
                "Rotate user agents and add random delays"
            ]
        }

def create_custom_scraping_session(query: str, urls: List[str], selectors: Dict[str, str], max_pages: int = 2):
    """Create a custom multi-page scraping session with active agent participation"""

    async def run_custom_session():
        system = MultiAgentWebScrapingSystem()

        print("Starting agent coordination...")

        # Simplified agent workflow with actual scraping integration
        coordinator_message = f"""
        MasterCoordinator: Initiating multi-page scraping task for query: "{query}"

        Task Details:
        - Target URLs: {len(urls)} starting points
        - Max pages per URL: {max_pages}
        - Data to extract: {list(selectors.keys())}

        SearchStrategy: Please analyze the query and recommend scraping approach.
        GenericScraper: Please confirm readiness for multi-page scraping.
        DataProcessor: Please prepare for data cleaning and deduplication.
        QualityAssurance: Please prepare quality metrics for final assessment.
        """

        print(coordinator_message)

        # Strategy analysis
        strategy_message = f"""
        SearchStrategy: Analysis complete for query "{query}"

        Recommended approach:
        1. Multi-page scraping with pagination detection
        2. Enhanced selectors for business directory sites
        3. Data extraction focus: {', '.join(selectors.keys())}
        4. Quality checks at each page
        5. Duplicate prevention across pages

        GenericScraper: Proceed with scraping using provided selectors.
        """

        print(strategy_message)

        # Execute actual scraping with agent integration
        print(" GenericScraper: Starting multi-page scraping...")
        scraping_results = await system.scrape_with_agents(query, urls, selectors, max_pages)

        # Final processing
        print(" DataProcessor: Processing complete dataset...")
        processed_data = system.process_final_results(scraping_results)

        print(" QualityAssurance: Performing final quality assessment...")
        quality_report = system.generate_quality_report(scraping_results)

        final_message = f"""
        MasterCoordinator: Multi-page scraping task completed successfully!

        Final Results:
        - Total pages scraped: {len(scraping_results)}
        - Total businesses found: {sum(len(result.get('business_name', [])) for result in scraping_results)}
        - Success rate: {(len(scraping_results) - len([r for r in scraping_results if 'error' in r])) / len(scraping_results) * 100:.1f}%
        - Data quality score: {system.calculate_quality_score(scraping_results)}/10

        Task completed with all agents participating in the workflow.
        """

        print(final_message)

        return {
            "query": query,
            "agent_workflow": [
                coordinator_message,
                strategy_message,
                final_message
            ],
            "scraping_results": scraping_results,
            "processed_data": processed_data,
            "quality_report": quality_report,
            "summary": {
                "total_pages_scraped": len(scraping_results),
                "total_businesses_found": sum(len(result.get('business_name', [])) for result in scraping_results),
                "pages_with_errors": len([r for r in scraping_results if 'error' in r]),
                "success_rate": (len(scraping_results) - len([r for r in scraping_results if 'error' in r])) / len(scraping_results) * 100 if scraping_results else 0,
                "data_quality_score": system.calculate_quality_score(scraping_results)
            },
            "timestamp": datetime.now().isoformat()
        }

    return run_custom_session()


In [84]:
# Example usage for Just Dial scraping
urls = ["https://quotes.toscrape.com/page/1/"]

image_selector_string = "article img, main img, .content img, .post-content img, .entry-content img, figure img, .featured-image img, .wp-post-image, .post-thumbnail img, .card__image, img[class*='image'], img[class*='photo']"

selectors = {
    #"business_name": ".jcn a, .resultbox .rsltTitle",
    "phone_number": ".contact-info .mobile, .ph",
    "address": ".address, .addr",
    "image":image_selector_string,
    "rating": ".rating, .star-rating",
    "title": "title",
    "headings": "h1, h2, h3",
    "paragraphs": "p",
    "code_blocks": "pre, code",
     "quote_text": "span.text",
    "author": "small.author",
    "tags": ".tags .tag",
}

In [85]:
# To run the scraper:
result = await create_custom_scraping_session(
    "Scrap the data from given website",
    urls=urls,
    selectors=selectors,
    max_pages=2
)

Starting agent coordination...

        MasterCoordinator: Initiating multi-page scraping task for query: "Scrap the data from given website"
        
        Task Details:
        - Target URLs: 1 starting points
        - Max pages per URL: 2
        - Data to extract: ['phone_number', 'address', 'image', 'rating', 'title', 'headings', 'paragraphs', 'code_blocks', 'quote_text', 'author', 'tags']
        
        SearchStrategy: Please analyze the query and recommend scraping approach.
        GenericScraper: Please confirm readiness for multi-page scraping.
        DataProcessor: Please prepare for data cleaning and deduplication.
        QualityAssurance: Please prepare quality metrics for final assessment.
        

        SearchStrategy: Analysis complete for query "Scrap the data from given website"
        
        Recommended approach:
        1. Multi-page scraping with pagination detection
        2. Enhanced selectors for business directory sites
        3. Data extraction 

In [86]:
result

{'query': 'Scrap the data from given website',
 'agent_workflow': ['\n        MasterCoordinator: Initiating multi-page scraping task for query: "Scrap the data from given website"\n        \n        Task Details:\n        - Target URLs: 1 starting points\n        - Max pages per URL: 2\n        - Data to extract: [\'phone_number\', \'address\', \'image\', \'rating\', \'title\', \'headings\', \'paragraphs\', \'code_blocks\', \'quote_text\', \'author\', \'tags\']\n        \n        SearchStrategy: Please analyze the query and recommend scraping approach.\n        GenericScraper: Please confirm readiness for multi-page scraping.\n        DataProcessor: Please prepare for data cleaning and deduplication.\n        QualityAssurance: Please prepare quality metrics for final assessment.\n        ',
  '\n        SearchStrategy: Analysis complete for query "Scrap the data from given website"\n        \n        Recommended approach:\n        1. Multi-page scraping with pagination detection\n     

In [79]:
for res in result:
  print(res)
  import time
  time.sleep(1)

query
agent_workflow
scraping_results
processed_data
quality_report
summary
timestamp


In [87]:
result['processed_data']

{'businesses': [],
 'statistics': {'total_pages': 2,
  'total_businesses': 0,
  'unique_phone_numbers': 0,
  'unique_addresses': 0}}

In [83]:
###ALL

In [ ]:
import asyncio
import json
import time
from typing import Dict, List, Any
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright
import autogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import re
from urllib.parse import urljoin, urlparse
import os

# --- Configuration ---
# Note: It's safer to use environment variables for API keys.
# For example: os.getenv("OPENAI_API_KEY")
config_list = [
    {
        "model": "gpt-4",
        "api_key": your_api_key,  # Replace with your actual API key
        "base_url": "https://api.openai.com/v1",
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0.1,
    "timeout": 120,
}


# --- Web Scraper Class ---

class SimpleWebScraper:
    """A general-purpose web scraper using Playwright with multi-page support."""

    def __init__(self):
        self.browser = None
        self.context = None
        self.scraped_urls = set()

    async def setup_browser(self, headless=True):
        """Initializes the Playwright browser."""
        playwright = await async_playwright().start()
        self.browser = await playwright.chromium.launch(headless=headless)
        self.context = await self.browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
            viewport={"width": 1920, "height": 1080}
        )
        await self.context.route("**/*", lambda route: route.continue_())

    async def wait_for_page_load(self, page):
        """Waits for the page to reach a stable state."""
        try:
            await page.wait_for_load_state('networkidle', timeout=15000)
            await asyncio.sleep(2)  # Extra wait for any client-side rendering.
        except Exception:
            await asyncio.sleep(3)  # Fallback wait.

    async def extract_pagination_links(self, page, base_url):
        """Extracts pagination links using a list of common selectors."""
        pagination_links = []
        # A list of generic selectors that often match 'next' page links.
        pagination_selectors = [
            'a[rel="next"]', 'a[aria-label*="Next"]', 'a:has-text("Next")',
            '.pagination a', '.pager a', 'a.next', 'li.next a', '.page-numbers a'
        ]

        for selector in pagination_selectors:
            try:
                elements = await page.query_selector_all(selector)
                for element in elements:
                    href = await element.get_attribute('href')
                    if href:
                        full_url = urljoin(base_url, href)
                        if full_url not in self.scraped_urls:
                            pagination_links.append(full_url)
            except Exception as e:
                print(f"Selector error '{selector}': {e}")
        return list(set(pagination_links))

    async def scrape_page(self, url: str, selectors: Dict[str, str]) -> Dict[str, Any]:
        """Scrapes a single page based on the provided CSS selectors."""
        page = None
        try:
            page = await self.context.new_page()
            await page.goto(url, wait_until="domcontentloaded", timeout=30000)
            await self.wait_for_page_load(page)

            data = {"url": url, "scraped_at": datetime.now().isoformat()}
            all_items_data = []

            # Find the common containers for the items to scrape.
            # This logic assumes that the selectors will return lists of elements of similar length.
            scraped_elements = {}
            max_len = 0
            for key, selector in selectors.items():
                try:
                    elements = await page.query_selector_all(selector)
                    texts = [await el.text_content() for el in elements]
                    scraped_elements[key] = [text.strip() for text in texts if text]
                    if len(scraped_elements[key]) > max_len:
                        max_len = len(scraped_elements[key])
                except Exception as e:
                    print(f"Error scraping '{key}' with selector '{selector}': {e}")
                    scraped_elements[key] = []

            # Structure the data into a list of dictionaries, one for each item.
            for i in range(max_len):
                item_data = {}
                for key in selectors.keys():
                    if i < len(scraped_elements.get(key, [])):
                        item_data[key] = scraped_elements[key][i]
                    else:
                        item_data[key] = None # Use None for missing data
                all_items_data.append(item_data)

            data['items'] = all_items_data
            data["pagination_links"] = await self.extract_pagination_links(page, url)
            return data

        except Exception as e:
            print(f"Error scraping {url}: {e}")
            return {"url": url, "error": str(e)}
        finally:
            if page:
                await page.close()


    async def scrape_multiple_pages(self, start_url: str, selectors: Dict[str, str], max_pages: int = 10) -> List[Dict[str, Any]]:
        """Scrapes multiple pages by following pagination links."""
        all_results = []
        urls_to_scrape = [start_url]
        pages_scraped = 0

        while urls_to_scrape and pages_scraped < max_pages:
            current_url = urls_to_scrape.pop(0)
            if current_url in self.scraped_urls:
                continue

            print(f"📄 Scraping page {pages_scraped + 1}/{max_pages}: {current_url}")
            page_data = await self.scrape_page(current_url, selectors)
            all_results.append(page_data)

            self.scraped_urls.add(current_url)
            pages_scraped += 1

            if "pagination_links" in page_data:
                for link in page_data["pagination_links"]:
                    if link not in self.scraped_urls:
                        urls_to_scrape.append(link)

            await asyncio.sleep(2) # Politeness delay
        return all_results

    async def close(self):
        """Closes the browser."""
        if self.browser:
            await self.browser.close()


# --- Multi-Agent System ---

class MultiAgentWebScrapingSystem:
    """An agent-based system for orchestrating web scraping tasks."""

    def __init__(self):
        self.setup_agents()

    def setup_agents(self):
        """Creates the specialized agents."""
        self.coordinator = AssistantAgent(
            name="MasterCoordinator",
            system_message="You are the Master Coordinator. You analyze user requests, orchestrate the scraping workflow with other agents, and compile the final results. Your primary goal is to manage the process from start to finish.",
            llm_config=llm_config
        )
        self.strategy_agent = AssistantAgent(
            name="SearchStrategy",
            system_message="You are the Search Strategy Agent. Your role is to analyze user queries, identify target websites, and suggest CSS selectors for the required data elements. You provide a clear plan for the GenericScraper agent.",
            llm_config=llm_config
        )
        self.generic_scraper = AssistantAgent(
            name="GenericScraper",
            system_message="You are the Generic Web Scraper Agent. You execute scraping tasks using the provided URLs and selectors. You are responsible for handling navigation, multi-page scraping, and reporting the raw results or any issues.",
            llm_config=llm_config
        )
        self.data_processor = AssistantAgent(
            name="DataProcessor",
            system_message="You are the Data Processing Agent. You clean, structure, and standardize the raw scraped data. Your job is to remove duplicates, format the data correctly, and prepare it for final presentation.",
            llm_config=llm_config
        )
        self.qa_agent = AssistantAgent(
            name="QualityAssurance",
            system_message="You are the Quality Assurance Agent. You verify the accuracy and completeness of the processed data. You check for inconsistencies, validate data quality, and provide a final quality score.",
            llm_config=llm_config
        )
        self.user_proxy = UserProxyAgent(
            name="UserProxy",
            system_message="You are the user's proxy, facilitating communication and executing code.",
            code_execution_config={"work_dir": "scraping_work", "use_docker": False},
            human_input_mode="NEVER"
        )
        self.groupchat = GroupChat(
            agents=[self.coordinator, self.strategy_agent, self.generic_scraper, self.data_processor, self.qa_agent, self.user_proxy],
            messages=[],
            max_round=20,
            speaker_selection_method="auto"
        )
        self.manager = GroupChatManager(groupchat=self.groupchat, llm_config=llm_config)

    def process_final_results(self, results: List[Dict]) -> Dict:
        """Processes and structures the final scraping results."""
        processed_items = []
        stats = {
            "total_pages_scraped": len(results),
            "pages_with_errors": 0,
            "total_items_found": 0,
        }

        for page_result in results:
            if 'error' in page_result:
                stats["pages_with_errors"] += 1
                continue

            page_items = page_result.get('items', [])
            for item in page_items:
                # Add source URL to each item for traceability
                item['source_url'] = page_result.get('url')
                processed_items.append(item)

        stats["total_items_found"] = len(processed_items)

        # Create a DataFrame for easy viewing and saving
        df = pd.DataFrame(processed_items)

        # Save results to CSV
        output_path = "scraped_data.csv"
        df.to_csv(output_path, index=False)
        print(f"💾 Data saved to {output_path}")

        return {"summary_stats": stats, "data": df}

    async def run_scraping_task(self, query: str, urls: List[str], selectors: Dict[str, str], max_pages: int = 3):
        """Executes the full scraping and processing workflow."""
        scraper_instance = SimpleWebScraper()
        try:
            print("🚀 Initializing scraping task...")
            print(f"Coordinator: Task for query '{query}' received.")
            print(f"StrategyAgent: Using URLs: {urls} with selectors: {selectors.keys()}")

            await scraper_instance.setup_browser()

            all_results = []
            for url in urls:
                results = await scraper_instance.scrape_multiple_pages(url, selectors, max_pages)
                all_results.extend(results)

            print("DataProcessor: Cleaning and structuring the final data...")
            final_data = self.process_final_results(all_results)

            print("QualityAssurance: Final data summary:")
            print(json.dumps(final_data["summary_stats"], indent=2))

            print("\n✅ Scraping task completed successfully!")
            return final_data

        except Exception as e:
            print(f"An error occurred during the scraping task: {e}")
        finally:
            await scraper_instance.close()



In [92]:

    # --- Example Task: Scrape Blog Post Titles and Authors ---
task_query = "Scrape titles and authors from the first 2 pages of a tech blog."

target_urls = ["https://quotes.toscrape.com/page/1/"]
image_selector_string = "article img, main img, .content img, .post-content img, .entry-content img, figure img, .featured-image img, .wp-post-image, .post-thumbnail img, .card__image, img[class*='image'], img[class*='photo']"


target_selectors = {
    #"business_name": ".jcn a, .resultbox .rsltTitle",
    "title": "h2.post-block__title a",
    "author": "span.river-byline__authors a",
    "phone_number": ".contact-info .mobile, .ph",
    "address": ".address, .addr",
    "image": image_selector_string,
    "rating": ".rating, .star-rating",
    "title": "title",
    "headings": "h1, h2, h3",
    "paragraphs": "p",
    "code_blocks": "pre, code",
     "quote_text": "span.text",
    "author": "small.author",
    "tags": ".tags .tag",
}
max_pages_to_scrape = 2

system = MultiAgentWebScrapingSystem()
res=await system.run_scraping_task(
        query=task_query,
        urls=target_urls,
        selectors=target_selectors,
        max_pages=max_pages_to_scrape
)

🚀 Initializing scraping task...
Coordinator: Task for query 'Scrape titles and authors from the first 2 pages of a tech blog.' received.
StrategyAgent: Using URLs: ['https://quotes.toscrape.com/page/1/'] with selectors: dict_keys(['title', 'author', 'phone_number', 'address', 'image', 'rating', 'headings', 'paragraphs', 'code_blocks', 'quote_text', 'tags'])
📄 Scraping page 1/2: https://quotes.toscrape.com/page/1/
📄 Scraping page 2/2: https://quotes.toscrape.com/page/2/
DataProcessor: Cleaning and structuring the final data...
💾 Data saved to scraped_data.csv
QualityAssurance: Final data summary:
{
  "total_pages_scraped": 2,
  "pages_with_errors": 0,
  "total_items_found": 68
}

✅ Scraping task completed successfully!


In [93]:
res['data']

,title,author,phone_number,address,image,rating,headings,paragraphs,code_blocks,quote_text,tags,source_url
0,Quotes to Scrape,Albert Einstein,None,None,None,None,Quotes to Scrape,Login,None,“The world as we have created it is a process ...,change,https://quotes.toscrape.com/page/1/
1,None,J.K. Rowling,None,None,None,None,Top Ten tags,Quotes by: GoodReads.com,None,"“It is our choices, Harry, that show what we t...",deep-thoughts,https://quotes.toscrape.com/page/1/
2,None,Albert Einstein,None,None,None,None,None,Made with ❤ by Zyte,None,“There are only two ways to live your life. On...,thinking,https://quotes.toscrape.com/page/1/
3,None,Jane Austen,None,None,None,None,None,None,None,"“The person, be it gentleman or lady, who has ...",world,https://quotes.toscrape.com/page/1/
4,None,Marilyn Monroe,None,None,None,None,None,None,None,"“Imperfection is beauty, madness is genius and...",abilities,https://quotes.toscrape.com/page/1/
...,...,...,...,...,...,...,...,...,...,...,...,...
63,None,None,None,None,None,None,None,None,None,None,fate,https://quotes.toscrape.com/page/2/
64,None,None,None,None,None,None,None,None,None,None,life,https://quotes.toscrape.com/page/2/
65,None,None,None,None,None,None,None,None,None,None,misattributed-john-lennon,https://quotes.toscrape.com/page/2/
66,None,None,None,None,None,None,None,None,None,None,planning,https://quotes.toscrape.com/page/2/
